In [1]:
import taxcalc as tc
import pandas as pd
from bokeh.io import show, output_notebook

In [2]:
recs = tc.Records.cps_constructor()

In [3]:
pol = tc.Policy()
calc1 = tc.Calculator(policy=pol, records=recs)

In [4]:
CYR = 2024

In [5]:
calc1.advance_to_year(CYR)
calc1.calc_all()
itax_rev1 = calc1.weighted_total('iitax')

In [6]:
reform_filename = './docs/recipes/_static/reform_WZ.json'
params = tc.Calculator.read_json_param_objects(reform_filename, None)

In [7]:
pol.implement_reform(params['policy'])
calc2 = tc.Calculator(policy=pol, records=recs)

In [8]:
calc2.advance_to_year(CYR)
calc2.calc_all()
itax_rev2 = calc2.weighted_total('iitax')

In [9]:
print('{}_CLP_itax_rev($B)= {:.3f}'.format(CYR, itax_rev1 * 1e-9))
print('{}_REF_itax_rev($B)= {:.3f}'.format(CYR, itax_rev2 * 1e-9))

2024_CLP_itax_rev($B)= 1813.745
2024_REF_itax_rev($B)= 1824.725


In [10]:
# Aggregate diagnostic tables for CYR.
clp_diagnostic_table = calc1.diagnostic_table(1)
ref_diagnostic_table = calc2.diagnostic_table(1)

# Income-tax distribution for CYR with CLP and REF results side-by-side.
dist_table1, dist_table2 = calc1.distribution_tables(calc2, 'weighted_deciles')
assert isinstance(dist_table1, pd.DataFrame)
assert isinstance(dist_table2, pd.DataFrame)
dist_extract = pd.DataFrame()
dist_extract['funits(#m)'] = dist_table1['count']
dist_extract['itax1($b)'] = dist_table1['iitax']
dist_extract['itax2($b)'] = dist_table2['iitax']
dist_extract['aftertax_inc1($b)'] = dist_table1['aftertax_income']
dist_extract['aftertax_inc2($b)'] = dist_table2['aftertax_income']

# Income-tax difference table by expanded-income decile for CYR.
diff_table = calc1.difference_table(calc2, 'weighted_deciles', 'iitax')
assert isinstance(diff_table, pd.DataFrame)
diff_extract = pd.DataFrame()
dif_colnames = ['count', 'tot_change', 'mean', 'pc_aftertaxinc']
ext_colnames = ['funits(#m)', 'agg_diff($b)', 'mean_diff($)', 'aftertaxinc_diff(%)']
for dname, ename in zip(dif_colnames, ext_colnames):
    diff_extract[ename] = diff_table[dname]

In [11]:
fig = calc1.pch_graph(calc2)
output_notebook()
show(fig)

Loading BokehJS ...

SerializationError: can't serialize <class 'range'>

In [11]:
clp_diagnostic_table

,2024
Returns (#m),214.880
AGI ($b),15077.541
Itemizers (#m),29.410
Itemized Deduction ($b),914.903
Standard Deduction Filers (#m),185.460
Standard Deduction ($b),3805.070
Personal Exemption ($b),0.000
Taxable Income ($b),11424.368
Regular Tax ($b),2016.056
AMT Income ($b),14379.040
